In [1]:
import sys
sys.dont_write_bytecode=True

import os

import torch

from modules import VoxelDataset, LatentDataset
from modules import Diffusion
from modules import Encoder
from modules import Decoder
from modules import VanillaDiffusionSampler
from modules import KEulerDiffusionSampler
from modules import plot_part_by_voxel_coords, plot_objt_by_latents

from torch.utils.data import DataLoader
from torch.backends import cudnn

from tqdm import tqdm

cudnn.benchmark = True

device = 'cuda'

In [2]:
pram = {
    'data_dir_pth': './dataset/chair_voxel_data_remove_duplicate',
    'part_counts_npy_pth': './dataset/each_chair_parts_count_remove_duplicate.npy',
    'outlier_objt_indices_npy_pth': './dataset/outlier_objt_indices.npy',
    'latents_remove_duplicate_npy_pth': './dataset/latents_remove_duplicate.npy',
    'batch_size': 1,
    'voxel_map_shape': (128, 128, 128),
    'designate_num_objts': 1,
    'train_test_split_ratio_train': 1,
    'latent_dim': (64, 64),
    'beta_start': 1e-4,
    'beta_end': 2e-2,
    'training_steps': 1000,
    'inference_steps': 30,
    'warmup': 10
}

In [3]:
diffusion = Diffusion().to(device)

saved_ckpt = torch.load('./models/diffusion_saved.pt')

diffusion.load_state_dict(saved_ckpt['diffusion_state_dict'])

<All keys matched successfully>

In [4]:
# vanilla_sampler = VanillaDiffusionSampler(diffusion,
#                                           pram['beta_start'],
#                                           pram['beta_end'],
#                                           pram['training_steps']).to(device)

In [5]:
euler_sampler = KEulerDiffusionSampler(diffusion,
                                       pram['beta_start'],
                                       pram['beta_end'],
                                       pram['inference_steps'],
                                       pram['training_steps']).to(device)

In [6]:
# x_T = torch.randn((pram['batch_size'], 1, *pram['latent_dim']), device=device)

# x_t = x_T
# for time_step in tqdm(reversed(range(pram['training_steps']))):
#     x_t = vanilla_sampler(x_t, time_step)
# x_0 = x_t

In [7]:
x_T = torch.randn((pram['batch_size'], 1, *pram['latent_dim']), device=device)
x_T = x_T * euler_sampler.initial_scale

for time_step in tqdm(range(pram['inference_steps'])):
    x_T = euler_sampler(x_T, time_step)
x_0 = x_T

 50%|█████     | 15/30 [00:20<00:20,  1.38s/it]


KeyboardInterrupt: 

In [ ]:
decoder = Decoder(pram['latent_dim'], vd_ch=8).to(device)

saved_ckpt = torch.load('./models/encoder_decoder_8.pt')

decoder.load_state_dict(saved_ckpt['decoder_state_dict'])

In [ ]:
pred = torch.sigmoid(decoder(x_0))

torch.max(pred)

In [ ]:
(pred > 0.5).nonzero().shape

In [ ]:
voxel_coords = (pred > 0.5).nonzero()[:, 2:]
plot_part_by_voxel_coords(voxel_coords)